In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
# from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import datasets, transforms as T

import random, os, pathlib, time
from tqdm import tqdm

In [ ]:
device = torch.device("cuda:0")
# device = torch.device("cpu")

In [ ]:
from tqdm import tqdm
import os, time, sys
import json

In [ ]:
import dtnnlib as dtnn

In [ ]:
mnist_transform = T.Compose([
    T.ToTensor(),
    T.Normalize(
        mean=[0.5,],
        std=[0.5,],
    ),
])

############################
#### Choose Dataset Here ###

# train_dataset = datasets.FashionMNIST(root="data/", train=True, download=True, transform=mnist_transform)
# test_dataset = datasets.FashionMNIST(root="data/", train=False, download=True, transform=mnist_transform)

train_dataset = datasets.MNIST(root="data/", train=True, download=True, transform=mnist_transform)
test_dataset = datasets.MNIST(root="data/", train=False, download=True, transform=mnist_transform)

In [ ]:
batch_size = 50
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

In [ ]:
for xx, yy in train_loader:
    xx, yy = xx.to(device), yy.to(device)
    print(xx.shape, yy.shape)
    break

In [ ]:
h = 10
model = nn.Sequential(
            dtnn.DistanceTransform_MinExp(784, h),
            nn.LeakyReLU(),
            nn.Linear(h, 10))

In [ ]:
model.to(device)

In [ ]:
# model[0].set_centroid_to_data(train_loader)
# model[0].set_centroid_to_data_randomly(train_loader)

## Randomly

In [ ]:
N = model[0].centers.shape[0]
new_center = []
new_labels = []
count = 0
for i, (xx, yy) in enumerate(train_loader):
    xx = xx.reshape(-1, model[0].input_dim).to(model[0].centers.device)
    if count+xx.shape[0] < N:
        new_center.append(xx)
        new_labels.append(yy)
        count += xx.shape[0]
    elif count >= N:
        break
    else:
        new_center.append(xx[:N-count])
        new_labels.append(yy[:N-count])
        count = N
        break
        
new_center = torch.cat(new_center, dim=0)
new_labels = torch.cat(new_labels, dim=0)

In [ ]:
new_center.shape, new_labels.shape

## Set


In [ ]:
weights = torch.zeros(len(new_labels), 10)
for i in range(len(new_labels)):
    weights[i, new_labels[i]] = 1.

In [ ]:
weights.shape

In [ ]:
model[0].centers.data = new_center.to(model[0].centers.device)
model[-1].weight.data = weights.t().to(model[-1].weight.data)

In [ ]:
best_acc = -1
def test(epoch, model):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
#         for batch_idx, (inputs, targets) in enumerate(tqdm(test_loader)):
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device).view(-1, 28*28), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
    print(f"[Test] {epoch} Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    
    acc = 100.*correct/total
    return acc

In [ ]:
model.eval()

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
test_acc = test(0, model)
test_acc

## Benchmark with stats

In [ ]:
HIDDEN_UNITS = [10, 50, 200, 1000, 5000, 20000]

seed = 2023
np.random.seed(seed)
SEEDS = np.random.randint(0, high=9999, size=20)
SEEDS

In [ ]:
test_accuracy = {h:[] for h in HIDDEN_UNITS}
test_accuracy

In [ ]:
def get_centers_and_labels(data_loader, N):
    new_center = []
    new_labels = []
    count = 0
    for i, (xx, yy) in enumerate(data_loader):
        xx = xx.reshape(-1, model[0].input_dim).to(model[0].centers.device)
        if count+xx.shape[0] < N:
            new_center.append(xx)
            new_labels.append(yy)
            count += xx.shape[0]
        elif count >= N:
            break
        else:
            new_center.append(xx[:N-count])
            new_labels.append(yy[:N-count])
            count = N
            break

    new_center = torch.cat(new_center, dim=0)
    new_labels = torch.cat(new_labels, dim=0)
    return new_center, new_labels

In [ ]:
batch_size = 50
for h in HIDDEN_UNITS:
    print(f"Experiment for Hidden units: {h}")
    for seed in tqdm(SEEDS):
        seed = int(seed)
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        
        train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
        test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)
        
        model = nn.Sequential(
                    dtnn.DistanceTransform_MinExp(784, h),
                    nn.LeakyReLU(),
                    nn.Linear(h, 10)).to(device)
        
        new_center, new_labels = get_centers_and_labels(train_loader, h)
        weights = torch.zeros(len(new_labels), 10)
        for i in range(len(new_labels)):
            weights[i, new_labels[i]] = 1.
            
        model[0].centers.data = new_center.to(model[0].centers.device)
        model[-1].weight.data = weights.t().to(model[-1].weight.data)
        model.eval()
        
        test_acc = test(0, model)
        test_accuracy[h].append(test_acc)

In [ ]:
print(f"H \tMean \t\tSTD \tMAX")
for k, v in test_accuracy.items():
#     print(k, v)
    print(f"{k} \t{np.mean(v):.4f} \t{np.std(v):.4f} \t{np.max(v)}")